In [28]:
def function(folder_name):
    import pandas as pd

    destination = "/Users/NikiAthanasiadou/Datasets/BRFSS_LANDLINES/"+folder_name+".XPT"
    b11 = pd.read_sas(destination)
    #accomodate variations in field names (known)
    if "_AGE80" in list(b11.columns):
        b11 = b11[['_STATE', '_AGE80', 'EDUCA','DIABETE3', '_RFBMI5']]
    else:
        b11 = b11[['_STATE', 'AGE', 'EDUCA', 'DIABETE3', '_RFBMI5']]
    #Harmonize column names
    b11.columns = ['_STATE', 'AGE', 'EDUCA', 'DIABETE3', '_RFBMI5']
    
    #From age to age range (categorical)
    b11['AGE'][b11['AGE']==7] = 'NA'
    b11['AGE'][b11['AGE']==9] = 'NA'
    b11['AGE'][b11['AGE']<25] = 'Age 18-24'
    b11['AGE'][b11['AGE']<35] = 'Age 25-34'
    b11['AGE'][b11['AGE']<44] = 'Age 35-44'
    b11['AGE'][b11['AGE']<54] = 'Age 45-54'
    b11['AGE'][b11['AGE']<64] = 'Age 55-64'
    b11['AGE'][b11['AGE'] <200] = 'Age 65 or older'

#Compress identical rows
    b11 = b11.groupby(b11.columns.tolist(), as_index=False).size().reset_index()

#Make categories human-readable
    states = {1:'Alabama', 2:'Alaska',4:'Arizona',5:'Arkansas',6:'California',8:'Colorado',9:'Connecticut',10:'Delaware',11:'District of Columbia',12:'Florida',13:'Georgia',15:'Hawaii',16:'Idaho',17:'Illinois',18:'Indiana',19:'Iowa',20:'Kansas',21:'Kentucky',22:'Louisiana',23:'Maine',24:'Maryland',25:'Massachusetts',26:'Michigan',27:'Minnesota',28:'Mississippi',29:'Missouri',30:'Montana',31:'Nebraska',32:'Nevada',33:'New Hampshire',34:'New Jersey',35:'New Mexico',36:'New York',37:'North Carolina',38:'North Dakota',39:'Ohio',40:'Oklahoma',41:'Oregon',42:'Pennsylvania',44:'Rhode Island',45:'South Carolina',46:'South Dakota',47:'Tennessee',48:'Texas',49:'Utah',50:'Vermont', 51:'Virginia', 53:'Washington',54:'West Virginia',55:'Wisconsin',56:'Wyoming', 66:'NA', 72:'NA'}
    education = {1:'No school',2:'Elementary school',3:'Some high school',4:'High school graduate',5:'Some college or techical school',6:'College graduate',9:'NA'}
    diabetes  = {1:1,2:0, 3:0, 4:1,7:"NA",9:"NA"}
    bmi = {1:0, 2:1, 9:"NA" }
    a = []
    b = []
    c = []
    d = []
    for i in range(len(b11['AGE'])):
# Fix the state names
        a.append(states[b11['_STATE'] [i]])
#Fix the education level
        b.append(education[b11['EDUCA'] [i]])
#Fix diabetes
        c.append(diabetes[b11['DIABETE3'][i]])
#Fix bmi
        d.append(bmi[b11['_RFBMI5'][i]])
    b11['_STATE'] = a
    b11['EDUCA'] =  b
    b11['DIABETE3'] = c
    b11['_RFBMI5'] = d
    
    b11.columns = ['State', 'Age', 'Education level', 'Diabetes', 'BMI', 'Count']
## By following the method above (and adding the relevant columns at the begining) I can add anything
    
    
#Add column stating if either BMI or Diabetes are a Yes. If one of the two is NA, all is NA
    Both = []
    for i in range(len(b11['Age'])):
        if (b11['BMI'][i]==1) & (b11['Diabetes'][i]==1):
            Both.append(1)
        elif (b11['BMI'][i]=="NA") ^ (b11['Diabetes'][i]=="NA"):
            Both.append("NA")
        else :
            Both.append(0)
    b11['Both'] = pd.Series(Both, index=b11.index) 
        
#fix column names
   
    return(b11)

In [29]:
#Only run once
land2011 = function("LLCP2011")
land2012 = function("LAND2012")
land2013 = function("LAND2013")
land2014 = function("LAND2014")

In [30]:
land2011.to_csv('land2011.txt')
land2012.to_csv('land2012.txt')
land2013.to_csv('land2013.txt')
land2014.to_csv('land2014.txt')